In [40]:
import pandas as pd
import plotly.express as px

In [2]:
resilience = pd.read_pickle('../Data/Resilience/final.pkl')
emag       = pd.read_pickle('../Data/Emagazine/emag_final.pkl')
icn        = pd.read_pickle('../Data/Inside Climate News/icn.pkl')
grist      = pd.read_pickle('../Data/Grist/grist.pkl')

In [13]:
emag['url'] = emag.link
resilience['source'] = 'Resilience'
emag['source'] = 'EMagazine'
icn['source'] = 'InsideClimateNews'
grist['source'] = 'Grist'

In [24]:
emag['date'] = pd.to_datetime(emag.date)
icn['date'] = pd.to_datetime(icn.date)
grist['date'] = pd.to_datetime(grist.date)

In [25]:
stack = pd.concat([resilience, emag, icn, grist])[['source', 'url', 'title', 'date', 'author', 'text']]

In [35]:
stack = stack.copy()[~stack.text.isna()]

In [33]:
stack.source.value_counts()

Resilience           33370
Grist                22566
EMagazine             7915
InsideClimateNews     5742
Name: source, dtype: int64

In [36]:
stack['year'] = stack.date.dt.year

In [49]:
fig = px.line(stack.groupby(['source', 'year']).agg({'url':'count'}).reset_index(),
              x='year', y='url', color='source',
              labels={'year':'Year', 'url':'Articles Published', 'source':'Source'},
              title='Environmental Discourse: Number of Articles Published by Source')

fig.update_layout({
     'font_family':'Times New Roman',
     'font_color':'black',
     'plot_bgcolor':'rgba(0, 0, 0, 0)',

     'legend':dict(
            yanchor="top",
            xanchor="left",
            y=0.9,
            x=0.05
        ),
     'yaxis': dict(
         mirror=True,
         ticks='outside',
         showline=True
     ),
     'xaxis': dict(
         mirror=True,
         ticks='outside',
         showline=True
     )

    }
)
fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')

In [58]:
stack['wordcount'] = stack.text.apply(lambda x: len(x.split()))

In [71]:
fig = px.line(stack.groupby(['source', 'year']).agg({'wordcount':'mean'}).reset_index(),
              x='year', y='wordcount', color='source',
              labels={'year':'Year', 'wordcount':'Average Word Count', 'source':'Source'},
              title='Environmental Discourse: Words per Article by Source')

fig.update_layout({
     'font_family':'Times New Roman',
     'font_color':'black',
     'plot_bgcolor':'rgba(0, 0, 0, 0)',

     'legend':dict(
            yanchor="top",
            xanchor="right",
            y=0.9,
            x=0.95
        ),
     'yaxis': dict(
         mirror=True,
         ticks='outside',
         showline=True
     ),
     'xaxis': dict(
         mirror=True,
         ticks='outside',
         showline=True
     )

    }
)
fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')

In [69]:
stack['cleaned_wordlist'] = stack.text.apply(
    lambda x: x.lower().split()
)

stack['science_count'] = stack.cleaned_wordlist.apply(
    lambda x: x.count('science')
)

In [70]:
stack.head()

,source,url,title,date,author,text,year,wordcount,cleaned_wordlist,science_count
21,Resilience,https://www.resilience.org/stories/2021-07-12/...,Scientists Call Northwest Heatwave the ‘Most E...,2021-07-12,Jake Johnson,Ed. note: This piece was originally published ...,2021.0,635,"[ed., note:, this, piece, was, originally, pub...",0
22,Resilience,https://www.resilience.org/stories/2021-07-12/...,Letter From The Farm | Welcome To The Burren,2021-07-12,Shane Casey,Ed. note: This article first appeared on ARC20...,2021.0,1165,"[ed., note:, this, article, first, appeared, o...",1
23,Resilience,https://www.resilience.org/stories/2021-07-12/...,"Wisdom Traditions, Science and the Search for ...",2021-07-12,David Bollier,Jeremy Lent has taken on an audacious task for...,2021.0,614,"[jeremy, lent, has, taken, on, an, audacious, ...",2
24,Resilience,https://www.resilience.org/stories/2021-07-12/...,Defending Beef: Excerpt,2021-07-12,Nicolette Hahn Niman,The following excerpt is from Nicolette Hahn N...,2021.0,1928,"[the, following, excerpt, is, from, nicolette,...",1
25,Resilience,https://www.resilience.org/stories/2021-07-12/...,The show is over,2021-07-12,Greta Thunberg,Ed. note: This piece was originally published ...,2021.0,984,"[ed., note:, this, piece, was, originally, pub...",1


In [72]:
stack.to_pickle("../Data/Environmental Discourse/env.pkl")